In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio.merge import merge
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.warp import reproject, Resampling
import rasterstats as rs
import folium


In [ ]:
RegionVals = pd.read_pickle("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data/Processed/RegionVal.pkl")
RegionVals.keys()

In [ ]:
gdfs = []

# regions = RegionVals.keys()  # all regions
regions = ["S_Sierras", "N_Sierras"]  # just the two regions we want

for reg in regions:
    df = RegionVals[reg]
    df['region'] = reg  # add column to record original region
    gdfs.append(df)

In [ ]:
superset = pd.concat(gdfs)
superset = gpd.GeoDataFrame(superset, geometry=gpd.points_from_xy(superset.Long, superset.Lat, crs="EPSG:4326"))
superset.plot()

In [ ]:
# plot geotiff
# Open the file:
gran1 = rio.open("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/testh08v04_stolen.tif", 'r')
gran2 = rio.open("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/testh09v04_stolen.tif", 'r')
gran3 = rio.open("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/testh08v05_stolen.tif", 'r')

merged, transform = merge([gran1, gran2, gran3])
merged_meta = gran1.meta.copy()
merged_meta.update(
    {
        "driver": "GTiff",
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": transform,
    }
)

# Write the mosaic raster to disk:
with rio.open('/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/merged.tif', 'w', **merged_meta) as dst:
    dst.write(merged)

In [ ]:
with rio.open('/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/merged.tif') as sca:
    fig, ax = plt.subplots(figsize=(10, 10))
    ax = show(sca, ax=ax, cmap='viridis', title="Merged", alpha=0.5)
    superset.to_crs(sca.crs).plot(ax=ax, column='region', categorical=True, legend=True)
    plt.show()

In [ ]:
s = superset.to_crs(sca.crs)
s.crs

In [ ]:
with rio.open('/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/merged.tif') as sca:
# sca = rio.open('/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/merged.tif')
    buffered = gpd.GeoDataFrame(superset)  # TODO do we need a copy here?
    buffered['geometry'] = superset.to_crs(sca.crs).buffer(200)  # 200 meter buffer

    stats = rs.zonal_stats(buffered, sca.read(1), affine=sca.transform, stats=['mean'], geojson_out=False, copy_properties=True)

    superset["sca"] = [stat['mean'] for stat in stats]


In [ ]:
superset.to_file("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/testoutput2.geojson", driver='GeoJSON')